In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal, ndimage
from scipy.optimize import curve_fit

In [ ]:
pert_strength = 0.3
dir_name = 'T:/Team/Szewczyk/Data/2024-02-22/'

In [ ]:
current = pd.read_csv(dir_name+'example_period.csv')
prc = pd.read_csv(dir_name+'prc.csv')
prc['norm_response'] = prc.response/pert_strength

In [ ]:
def gaussian(x, h, m, s):
    d = np.minimum(np.abs(x-m), 1-np.abs(x-m))
    return h*np.exp(-d**2/(2*s))

def curve(x, h1, m1, s1, h2, m2, s2):
    return gaussian(x, h1, m1, s1) + gaussian(x, h2, m2, s2)

In [ ]:
initial_guess = (0.035, 0.18, 0.006, -0.1, 0.03, 0.002)
plt.figure()
plt.plot(prc.phase, curve(prc.phase, *initial_guess))
plt.scatter(prc.phase, prc.norm_response, marker='+', c='r')

In [ ]:
popt, popc = curve_fit(curve, prc.phase, prc.norm_response, initial_guess)

In [ ]:
def fitted_prc(x):
    return curve(x%1, *popt)

In [ ]:
plt.scatter(prc.phase, prc.norm_response, marker='+', c='r')
plt.plot(np.linspace(0, 1, 1000), curve(np.linspace(0, 1, 1000), *popt), zorder=10)

In [ ]:
plt.plot(current.phase, current.I)

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(current.phase, fitted_prc(current.phase))
ax1.set_ylabel('prc (blue)')
ax2.plot(current.phase, current.emsi_corrected, c='orange')
ax2.set_ylabel('emsi (orange)')
plt.show()

In [ ]:
effect = np.zeros_like(current.phase)
for i, d in enumerate(current.phase):
    effect[i] = np.sum(-fitted_prc(np.array(current.phase+d)%1) * (current.I-np.roll(current.I.to_numpy(), i)))/100

In [ ]:
def rolled_I(phi):
    return np.interp((current.phase+phi)%1, current.phase, current.I)

In [ ]:
def F(delta_phi):
    return np.sum((rolled_I(delta_phi)-current.I)*fitted_prc(current.phase))/current.phase.size

In [ ]:
effect = np.empty_like(current.phase)
for i, delta_phi in enumerate(current.phase):
    effect[i] = F(delta_phi)

In [ ]:
current.t.iloc[-1]

In [ ]:
plt.plot(current.phase, effect*current.t.iloc[-1])

In [ ]:
effect_pd = pd.DataFrame({
    'delta_phi'     : current.phase,
    'effect'        : effect*current.t.iloc[-1]
})
effect_pd.to_csv('effect_2LAs.csv')

In [ ]:
plt.plot(current.phase, effect + effect[::-1])
plt.plot(current.phase, effect-effect[::-1])